In [19]:
# XGBooosts
import  numpy as np
import pandas as pd
import mlflow
from datetime import datetime
import mlflow.xgboost
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier


In [5]:
# get data shuffle and split into train and test

data = pd.read_csv('../data/processed/asts.csv')
data = data.fillna(0)

# shuffle data
data = data.sample(frac=1).reset_index(drop=True)

In [11]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

target_columns = [
    "resistance_phenotype_not defined",                
    "resistance_phenotype_intermediate",               
    "resistance_phenotype_nonsusceptible",              
    "resistance_phenotype_resistant",                   
    "resistance_phenotype_susceptible",                 
    "resistance_phenotype_susceptible-dose dependent" 
]

# features the  rest of the columns
features = data.columns.difference(target_columns)

X = data[features]
y = data[target_columns]


# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Import additional required modules
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
# Clean feature names
def clean_feature_name(name):
    """Clean feature names to be compatible with XGBoost."""
    return str(name).replace('[', '_').replace(']', '_').replace('<', 'lt').replace('>', 'gt').replace('(', '_').replace(')', '_')

# Clean column names in X_train and X_test
X_train.columns = [clean_feature_name(col) for col in X_train.columns]
X_test.columns = [clean_feature_name(col) for col in X_test.columns]

# Create a multi-output classifier wrapper around XGBoost with specific parameters
model = MultiOutputClassifier(xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
))

# Use regular KFold instead of StratifiedKFold for multi-label
cv = KFold(n_splits=10, random_state=42, shuffle=True)

# Manual cross-validation for multi-label
scores = []
for train_idx, val_idx in cv.split(X_train):
    # Split data
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # Train and predict
    model.fit(X_train_fold, y_train_fold)
    y_pred = model.predict(X_val_fold)
    
    # Calculate accuracy
    fold_accuracy = accuracy_score(y_val_fold, y_pred)
    scores.append(fold_accuracy)

# Report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Accuracy: 0.982 (0.001)
